# Día 3: Limpieza de Datos y Visualización

**Introducción a Python para ML** | EAE Business School | 4 febrero 2026

En este notebook vamos a:
1. Cargar y explorar el dataset "sucio" de Barcelona
2. Identificar y limpiar problemas de calidad
3. Transformar datos
4. Usar GroupBy para agregaciones
5. Crear visualizaciones interactivas con Plotly Express

## Parte 1: Cargar Datos y Detectar Problemas

Vamos a trabajar con el dataset "sucio" que tiene problemas intencionales.

In [ ]:
import pandas as pd
import numpy as np

# Cargar dataset sucio
url = "https://raw.githubusercontent.com/ber2/eae-python/main/data/Houses_Barcelona_dirty.csv"
df = pd.read_csv(url)

print(f"Dimensiones: {df.shape}")
df.head()

### Inspección Inicial

Usemos `info()` y `describe()` para entender el dataset.

In [ ]:
# Info general
df.info()

In [ ]:
# Estadísticas descriptivas
df.describe()

**Pregunta**: ¿Qué problemas detectáis en los datos?

## Parte 2: Valores Faltantes

Identifiquemos cuántos valores faltan en cada columna.

In [ ]:
# Contar valores faltantes
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Columna': missing.index,
    'Faltantes': missing.values,
    'Porcentaje': missing_pct.values
})

# Solo mostrar columnas con valores faltantes
missing_df[missing_df['Faltantes'] > 0].sort_values('Faltantes', ascending=False)

### Estrategia para Valores Faltantes

Decisiones:
- **price**: ~10% faltante → rellenar con mediana
- **rooms**: ~15% faltante → rellenar con mediana (más robusta que media)
- **sqrmts**: ~10% faltante → rellenar con mediana

In [ ]:
# Rellenar valores faltantes
df['price'] = df['price'].fillna(df['price'].median())
df['rooms'] = df['rooms'].fillna(df['rooms'].median())
df['sqrmts'] = df['sqrmts'].fillna(df['sqrmts'].median())

# Verificar
print("Valores faltantes después de limpieza:")
print(df[['price', 'rooms', 'sqrmts']].isnull().sum())

## Parte 3: Duplicados

Busquemos filas duplicadas.

In [ ]:
# Detectar duplicados
duplicados = df.duplicated()
print(f"Filas duplicadas: {duplicados.sum()}")

# Ver algunas filas duplicadas
if duplicados.sum() > 0:
    print("\nEjemplo de duplicados:")
    df[df.duplicated(keep=False)].sort_values('id').head(10)

In [ ]:
# Eliminar duplicados
df_clean = df.drop_duplicates()
print(f"Filas antes: {len(df)}")
print(f"Filas después: {len(df_clean)}")
print(f"Eliminadas: {len(df) - len(df_clean)}")

df = df_clean.copy()

## Parte 4: Valores Imposibles

Busquemos valores que violan las reglas del negocio.

In [ ]:
# Precios negativos
neg_prices = df[df['price'] < 0]
print(f"Precios negativos: {len(neg_prices)}")
if len(neg_prices) > 0:
    print(neg_prices[['id', 'neighborhood', 'price', 'sqrmts']])

In [ ]:
# Habitaciones = 0 o muy altas
bad_rooms = df[(df['rooms'] == 0) | (df['rooms'] > 20)]
print(f"Habitaciones anómalas: {len(bad_rooms)}")
if len(bad_rooms) > 0:
    print(bad_rooms[['id', 'neighborhood', 'rooms', 'price']])

In [ ]:
# Metros cuadrados muy grandes
big_sqrmts = df[df['sqrmts'] > 500]
print(f"Propiedades >500m²: {len(big_sqrmts)}")
if len(big_sqrmts) > 0:
    print(big_sqrmts[['id', 'neighborhood', 'type', 'sqrmts', 'price']])

### Limpiar Valores Imposibles

Eliminaremos las filas con valores claramente erróneos.

In [ ]:
# Filtrar datos válidos
df_clean = df[
    (df['price'] > 0) &
    (df['price'] < 5_000_000) &  # Máximo razonable
    (df['rooms'] > 0) &
    (df['rooms'] <= 10) &  # Máximo razonable
    (df['sqrmts'] > 0) &
    (df['sqrmts'] < 500)  # Máximo razonable para pisos
]

print(f"Filas antes: {len(df)}")
print(f"Filas después: {len(df_clean)}")
print(f"Eliminadas: {len(df) - len(df_clean)}")

df = df_clean.copy()

## Parte 5: Transformación de Datos

Crear nuevas columnas útiles para el análisis.

In [ ]:
# Precio por metro cuadrado
df['price_per_sqm'] = (df['price'] / df['sqrmts']).round(0)

# Categorizar precios
df['price_category'] = pd.cut(df['price'],
                               bins=[0, 200000, 350000, 500000, 10000000],
                               labels=['Bajo', 'Medio', 'Alto', 'Premium'])

# Categorizar tamaño
df['size_category'] = pd.cut(df['sqrmts'],
                              bins=[0, 50, 80, 120, 500],
                              labels=['Pequeño', 'Mediano', 'Grande', 'Muy Grande'])

df[['neighborhood', 'price', 'sqrmts', 'price_per_sqm', 'price_category', 'size_category']].head(10)

## Parte 6: Operaciones GroupBy

Agregar datos por categorías para obtener insights.

In [ ]:
# Precio medio por barrio
precio_barrio = df.groupby('neighborhood')['price'].mean().sort_values(ascending=False)
print("Precio medio por barrio:")
print(precio_barrio)

In [ ]:
# Múltiples estadísticas por barrio
stats_barrio = df.groupby('neighborhood').agg({
    'price': ['mean', 'median', 'count'],
    'price_per_sqm': 'mean',
    'rooms': 'mean'
}).round(0)

stats_barrio.columns = ['_'.join(col).strip() for col in stats_barrio.columns.values]
stats_barrio.sort_values('price_mean', ascending=False)

In [ ]:
# Agrupar por barrio y tipo
precio_barrio_tipo = df.groupby(['neighborhood', 'type'])['price'].mean().reset_index()
precio_barrio_tipo.sort_values('price', ascending=False).head(15)

## Parte 7: Visualización con Plotly Express

Ahora vamos a crear visualizaciones interactivas.

In [ ]:
import plotly.express as px

print("Plotly Express importado correctamente")

### Scatter Plot: Precio vs Metros Cuadrados

In [ ]:
# Scatter básico
fig = px.scatter(df, x='sqrmts', y='price',
                 title='Relación entre Tamaño y Precio')
fig.show()

In [ ]:
# Scatter con color por barrio
fig = px.scatter(df, x='sqrmts', y='price', color='neighborhood',
                 title='Precio vs Tamaño por Barrio',
                 labels={'sqrmts': 'Metros Cuadrados', 'price': 'Precio (€)'},
                 hover_data=['type', 'rooms'])
fig.show()

### Bar Chart: Precio Medio por Barrio

In [ ]:
# Preparar datos
avg_price_by_neighborhood = df.groupby('neighborhood')['price'].mean().sort_values(ascending=False).reset_index()

# Bar chart
fig = px.bar(avg_price_by_neighborhood, 
             x='neighborhood', y='price',
             title='Precio Medio por Barrio',
             labels={'price': 'Precio Medio (€)', 'neighborhood': 'Barrio'})
fig.show()

### Histogram: Distribución de Precios

In [ ]:
# Histogram simple
fig = px.histogram(df, x='price', nbins=50,
                   title='Distribución de Precios',
                   labels={'price': 'Precio (€)'})
fig.show()

In [ ]:
# Histogram por tipo de propiedad
fig = px.histogram(df, x='price', color='type', nbins=40,
                   title='Distribución de Precios por Tipo',
                   labels={'price': 'Precio (€)'})
fig.show()

### Box Plot: Precios por Barrio

In [ ]:
# Box plot
fig = px.box(df, x='neighborhood', y='price',
             title='Distribución de Precios por Barrio',
             labels={'price': 'Precio (€)', 'neighborhood': 'Barrio'})
fig.show()

### Ejercicio: Crear Vuestra Visualización

Cread un scatter plot que muestre:
- Eje X: habitaciones
- Eje Y: precio por m²
- Color: barrio
- Información hover: tipo, metros cuadrados

In [ ]:
# Vuestra solución aquí


## Resumen del Notebook

**Lo que hemos practicado**:

✅ Detectar problemas de calidad de datos
✅ Manejar valores faltantes con `fillna()`
✅ Eliminar duplicados con `drop_duplicates()`
✅ Filtrar valores imposibles
✅ Crear columnas derivadas
✅ Usar GroupBy para agregaciones
✅ Crear visualizaciones interactivas con Plotly Express

**Mañana**: Estadística descriptiva y distribuciones de probabilidad